In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

['CONTENT_CATEGORY.csv',
 '.ipynb_checkpoints',
 'device_data.csv',
 'sampleSubmission.csv',
 'SITE_ID.csv',
 'PAGE.csv',
 'CONTENT_CATEGORY_TOP.csv',
 'conversiones.csv',
 'CONTENT_CATEGORY_BOTTOM.csv',
 'pageviews.csv']

## LOAD PAGE VIEWS INFO

In [2]:
col_user = 'USER_ID'
pageview = pd.read_csv("{}/pageviews.csv".format(subfolder), parse_dates=["FEC_EVENT"]).sort_values([col_user, "FEC_EVENT"])
pageview.shape

(17936934, 8)

In [3]:
null_verificator(pageview)

,0
msje,DATA LIMPIA DE NULOS


In [4]:
pageview.dtypes

FEC_EVENT                  datetime64[ns]
PAGE                                int64
CONTENT_CATEGORY                    int64
CONTENT_CATEGORY_TOP                int64
CONTENT_CATEGORY_BOTTOM             int64
SITE_ID                             int64
ON_SITE_SEARCH_TERM                 int64
USER_ID                             int64
dtype: object

In [5]:
pageview['month'] = pageview['FEC_EVENT'].dt.month

for col in pageview.columns:
    print(col, pageview[col].unique().shape[0])

FEC_EVENT 10496454
PAGE 1725
CONTENT_CATEGORY 65
CONTENT_CATEGORY_TOP 13
CONTENT_CATEGORY_BOTTOM 65
SITE_ID 4
ON_SITE_SEARCH_TERM 294
USER_ID 11676
month 12


In [6]:
def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0
    
pageview['trimestre'] = pageview['month'].apply(get_trimestre)
## del pageview['month']
pageview['trimestre'].unique()

array([1, 2, 3, 4])

In [7]:
pageview[['PAGE','CONTENT_CATEGORY','CONTENT_CATEGORY_TOP','CONTENT_CATEGORY_BOTTOM', 'SITE_ID', 'trimestre', 'month', 'ON_SITE_SEARCH_TERM']].corr()

,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,trimestre,month,ON_SITE_SEARCH_TERM
PAGE,1.000000,0.175382,0.141580,0.175382,0.040502,0.086931,0.089182,0.007231
CONTENT_CATEGORY,0.175382,1.000000,0.643041,1.000000,-0.207466,-0.076358,-0.079462,0.019454
CONTENT_CATEGORY_TOP,0.141580,0.643041,1.000000,0.643041,0.061494,0.024470,0.025535,0.025280
CONTENT_CATEGORY_BOTTOM,0.175382,1.000000,0.643041,1.000000,-0.207466,-0.076358,-0.079462,0.019454
SITE_ID,0.040502,-0.207466,0.061494,-0.207466,1.000000,0.141987,0.146864,-0.010708
trimestre,0.086931,-0.076358,0.024470,-0.076358,0.141987,1.000000,0.975801,-0.005437
month,0.089182,-0.079462,0.025535,-0.079462,0.146864,0.975801,1.000000,-0.006056
ON_SITE_SEARCH_TERM,0.007231,0.019454,0.025280,0.019454,-0.010708,-0.005437,-0.006056,1.000000


In [12]:
if 'CONTENT_CATEGORY_BOTTOM' in pageview.columns:
    del pageview['CONTENT_CATEGORY_BOTTOM']

if 'month' in pageview.columns:
    del pageview['month']

In [13]:
pageview.groupby(['trimestre']).count()[col_user]

trimestre
1    4861441
2    5677125
3    2497836
4    4900532
Name: USER_ID, dtype: int64

In [14]:
all_user = set(list(pageview[col_user].unique()))
len(all_user)

11676

In [15]:
def get_acumulate_data(data, cols_select=['PAGE', 'CONTENT_CATEGORY', 'ON_SITE_SEARCH_TERM', 'SITE_ID', col_user]):
    X_test = []
    for c in list(data[cols_select].columns)[:-1]:
        print("--> ", c)
        temp = pd.crosstab(data.USER_ID, data[c])
        temp.columns = [c + "_" + str(v) for v in temp.columns]
        
        X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
        del temp
    
    return pd.concat(X_test, axis=1)

In [17]:
X_test_k = get_acumulate_data(pageview)
all_columns = list(X_test_k.columns)

X_test_k_mini = X_test_k[list(X_test_k.columns)[:2]]

X_test_k = sp.sparse.csr_matrix(X_test_k.values)
len(all_columns), type(X_test_k)

-->  PAGE
-->  CONTENT_CATEGORY
-->  ON_SITE_SEARCH_TERM
-->  SITE_ID


(2088, scipy.sparse.csr.csr_matrix)

In [18]:
X_test_k_mini.head()

,PAGE_1,PAGE_2
USER_ID,,
0,0.028300,0.128440
1,0.103742,0.157271
2,0.032517,0.168074
3,0.005168,0.192765
4,0.010628,0.120451


In [19]:
data_view = []

for val in pageview['trimestre'].unique():
    page_trim = pageview[pageview['trimestre'] == val]
    unique_users = set(list(page_trim[col_user].unique()))
    
    print("Trimestre ", val, page_trim.shape, len(unique_users))
    # display(page_trim.head())
    
    X_test = get_acumulate_data(page_trim)

    # COMPLETE COLUMNS
    for col in set(all_columns) - set(list(X_test.columns)):
        X_test[col] = 0
    
    X_test = X_test[all_columns]
    
    #COMPLETE ROWS OF USERS
    temp0 = pd.DataFrame(0, index=list(all_user - unique_users), columns=X_test.columns)
    X_test = pd.concat([X_test, temp0]).sort_index()
    print("X_test post:: ", X_test.shape)
    
    data_view.append(X_test.copy())
    del X_test
    del page_trim


data_view = pd.concat(data_view, axis=0)
del pageview

display(data_view.head(10))
display(data_view.tail(10))
data_view.shape

Trimestre  1 (4861441, 8) 10936
-->  PAGE
-->  CONTENT_CATEGORY
-->  ON_SITE_SEARCH_TERM
-->  SITE_ID
X_test post::  (11676, 2088)
Trimestre  2 (5677125, 8) 11203
-->  PAGE
-->  CONTENT_CATEGORY
-->  ON_SITE_SEARCH_TERM
-->  SITE_ID
X_test post::  (11676, 2088)
Trimestre  3 (2497836, 8) 4765
-->  PAGE
-->  CONTENT_CATEGORY
-->  ON_SITE_SEARCH_TERM
-->  SITE_ID
X_test post::  (11676, 2088)
Trimestre  4 (4900532, 8) 11365
-->  PAGE
-->  CONTENT_CATEGORY
-->  ON_SITE_SEARCH_TERM
-->  SITE_ID
X_test post::  (11676, 2088)


,PAGE_1,PAGE_2,PAGE_3,PAGE_4,PAGE_5,PAGE_6,PAGE_7,PAGE_8,PAGE_9,PAGE_10,...,ON_SITE_SEARCH_TERM_289,ON_SITE_SEARCH_TERM_290,ON_SITE_SEARCH_TERM_291,ON_SITE_SEARCH_TERM_292,ON_SITE_SEARCH_TERM_293,ON_SITE_SEARCH_TERM_295,SITE_ID_1,SITE_ID_2,SITE_ID_3,SITE_ID_4
0,0.058121,0.067898,0.023900,0.008148,0.001086,0.006518,0.000543,0.000543,0.042368,0.177621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.065725,0.840304,0.093971,0.00000
1,0.125201,0.120385,0.027287,0.014446,0.001605,0.001605,0.000000,0.000000,0.046549,0.176565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.131621,0.775281,0.093098,0.00000
2,0.038186,0.157518,0.076372,0.021480,0.021480,0.000000,0.000000,0.000000,0.045346,0.145585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.245823,0.467780,0.286396,0.00000
3,0.000000,0.115607,0.023121,0.002890,0.002890,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.132948,0.820809,0.046243,0.00000
4,0.019516,0.096019,0.073380,0.000000,0.018735,0.021077,0.000000,0.000781,0.002342,0.270882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.044496,0.640125,0.302888,0.01249
5,0.000000,0.153757,0.036994,0.003468,0.004624,0.005780,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.138728,0.587283,0.273988,0.00000
6,0.000000,0.045139,0.045139,0.010417,0.006944,0.000000,0.000000,0.000000,0.010417,0.048611,...,0.0,0.0,0.0,0.0,0.0,0.0,0.010417,0.805556,0.184028,0.00000
7,0.097500,0.100000,0.015000,0.002500,0.000000,0.000000,0.000000,0.002500,0.127500,0.425000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.115000,0.862500,0.022500,0.00000
8,0.000000,0.164948,0.034364,0.010309,0.006873,0.000000,0.000000,0.000000,0.068729,0.271478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.134021,0.707904,0.158076,0.00000
9,0.005181,0.126943,0.119171,0.033679,0.012953,0.000000,0.000000,0.000000,0.010363,0.248705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.049223,0.660622,0.290155,0.00000


,PAGE_1,PAGE_2,PAGE_3,PAGE_4,PAGE_5,PAGE_6,PAGE_7,PAGE_8,PAGE_9,PAGE_10,...,ON_SITE_SEARCH_TERM_289,ON_SITE_SEARCH_TERM_290,ON_SITE_SEARCH_TERM_291,ON_SITE_SEARCH_TERM_292,ON_SITE_SEARCH_TERM_293,ON_SITE_SEARCH_TERM_295,SITE_ID_1,SITE_ID_2,SITE_ID_3,SITE_ID_4
11666,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11669,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11670,0.073171,0.195122,0.146341,0.048780,0.024390,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.073171,0.219512,0.707317,0.0
11671,0.019629,0.082879,0.162486,0.061069,0.034896,0.013086,0.0,0.005453,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.049073,0.117775,0.833152,0.0
11672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11673,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.086957,0.913043,0.0
11674,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
11675,0.000000,0.046512,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.139535,0.046512,0.813953,0.0


(46704, 2088)

In [20]:
data_view.shape

(46704, 2088)

## LOAD CONVERTIONS

In [21]:
convertion = pd.read_csv("{}/conversiones.csv".format(subfolder)).drop_duplicates()[['USER_ID','mes']].sort_values(['USER_ID','mes']).reset_index(drop=True)
display(convertion.head())
convertion.shape

,USER_ID,mes
0,13.0,8.0
1,22.0,12.0
2,40.0,1.0
3,57.0,6.0
4,92.0,3.0


(1564, 2)

In [22]:
for col in convertion.columns:
    convertion[col] = convertion[col].astype(int)

convertion.dtypes

USER_ID    int64
mes        int64
dtype: object

In [23]:
convertion['trimestre'] = convertion['mes'].apply(get_trimestre)
for col in convertion.columns:
    print(col, convertion[col].unique().shape)

del convertion['mes']
convertion.head()

USER_ID (1086,)
mes (12,)
trimestre (4,)


,USER_ID,trimestre
0,13,3
1,22,4
2,40,1
3,57,2
4,92,1


In [24]:
data_conv = []
col_target = 'TARGET'

for val in sorted(list(convertion['trimestre'].unique())):
    conv_trim = convertion[convertion['trimestre'] == val].drop_duplicates().reset_index(drop=True)
    unique_users_conv = set(list(conv_trim[col_user].unique()))
    conv_trim.index = conv_trim[col_user]
    
    conv_trim[col_target] = 1
    
    del conv_trim.index.name
    del conv_trim[col_user]
    del conv_trim['trimestre']

    temp0 = pd.DataFrame(0, index=list(all_user - unique_users_conv), columns=['TARGET'])
    X_test = pd.concat([conv_trim, temp0]).sort_index()

    print("X_test post:: ", X_test.shape)
    
    data_conv.append(X_test.copy())
    del X_test
    del conv_trim


data_conv = pd.concat(data_conv, axis=0)
del convertion

    
display(data_conv.head(20))
display(data_conv.tail(20))

X_test post::  (11676, 1)
X_test post::  (11676, 1)
X_test post::  (11676, 1)
X_test post::  (11676, 1)


,TARGET
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


,TARGET
11656,0
11657,0
11658,0
11659,0
11660,0
11661,0
11662,0
11663,0
11664,0
11665,0


In [25]:
data_conv.shape

(46704, 1)

In [26]:
data = pd.concat([data_view, data_conv], axis=1)
del data_view
del data_conv

data.head(10)

,PAGE_1,PAGE_2,PAGE_3,PAGE_4,PAGE_5,PAGE_6,PAGE_7,PAGE_8,PAGE_9,PAGE_10,...,ON_SITE_SEARCH_TERM_290,ON_SITE_SEARCH_TERM_291,ON_SITE_SEARCH_TERM_292,ON_SITE_SEARCH_TERM_293,ON_SITE_SEARCH_TERM_295,SITE_ID_1,SITE_ID_2,SITE_ID_3,SITE_ID_4,TARGET
0,0.058121,0.067898,0.023900,0.008148,0.001086,0.006518,0.000543,0.000543,0.042368,0.177621,...,0.0,0.0,0.0,0.0,0.0,0.065725,0.840304,0.093971,0.00000,0
1,0.125201,0.120385,0.027287,0.014446,0.001605,0.001605,0.000000,0.000000,0.046549,0.176565,...,0.0,0.0,0.0,0.0,0.0,0.131621,0.775281,0.093098,0.00000,0
2,0.038186,0.157518,0.076372,0.021480,0.021480,0.000000,0.000000,0.000000,0.045346,0.145585,...,0.0,0.0,0.0,0.0,0.0,0.245823,0.467780,0.286396,0.00000,0
3,0.000000,0.115607,0.023121,0.002890,0.002890,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.132948,0.820809,0.046243,0.00000,0
4,0.019516,0.096019,0.073380,0.000000,0.018735,0.021077,0.000000,0.000781,0.002342,0.270882,...,0.0,0.0,0.0,0.0,0.0,0.044496,0.640125,0.302888,0.01249,0
5,0.000000,0.153757,0.036994,0.003468,0.004624,0.005780,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138728,0.587283,0.273988,0.00000,0
6,0.000000,0.045139,0.045139,0.010417,0.006944,0.000000,0.000000,0.000000,0.010417,0.048611,...,0.0,0.0,0.0,0.0,0.0,0.010417,0.805556,0.184028,0.00000,0
7,0.097500,0.100000,0.015000,0.002500,0.000000,0.000000,0.000000,0.002500,0.127500,0.425000,...,0.0,0.0,0.0,0.0,0.0,0.115000,0.862500,0.022500,0.00000,0
8,0.000000,0.164948,0.034364,0.010309,0.006873,0.000000,0.000000,0.000000,0.068729,0.271478,...,0.0,0.0,0.0,0.0,0.0,0.134021,0.707904,0.158076,0.00000,0
9,0.005181,0.126943,0.119171,0.033679,0.012953,0.000000,0.000000,0.000000,0.010363,0.248705,...,0.0,0.0,0.0,0.0,0.0,0.049223,0.660622,0.290155,0.00000,0


In [27]:
data = data.reset_index()
data.head(10)

,index,PAGE_1,PAGE_2,PAGE_3,PAGE_4,PAGE_5,PAGE_6,PAGE_7,PAGE_8,PAGE_9,...,ON_SITE_SEARCH_TERM_290,ON_SITE_SEARCH_TERM_291,ON_SITE_SEARCH_TERM_292,ON_SITE_SEARCH_TERM_293,ON_SITE_SEARCH_TERM_295,SITE_ID_1,SITE_ID_2,SITE_ID_3,SITE_ID_4,TARGET
0,0,0.058121,0.067898,0.023900,0.008148,0.001086,0.006518,0.000543,0.000543,0.042368,...,0.0,0.0,0.0,0.0,0.0,0.065725,0.840304,0.093971,0.00000,0
1,1,0.125201,0.120385,0.027287,0.014446,0.001605,0.001605,0.000000,0.000000,0.046549,...,0.0,0.0,0.0,0.0,0.0,0.131621,0.775281,0.093098,0.00000,0
2,2,0.038186,0.157518,0.076372,0.021480,0.021480,0.000000,0.000000,0.000000,0.045346,...,0.0,0.0,0.0,0.0,0.0,0.245823,0.467780,0.286396,0.00000,0
3,3,0.000000,0.115607,0.023121,0.002890,0.002890,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.132948,0.820809,0.046243,0.00000,0
4,4,0.019516,0.096019,0.073380,0.000000,0.018735,0.021077,0.000000,0.000781,0.002342,...,0.0,0.0,0.0,0.0,0.0,0.044496,0.640125,0.302888,0.01249,0
5,5,0.000000,0.153757,0.036994,0.003468,0.004624,0.005780,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138728,0.587283,0.273988,0.00000,0
6,6,0.000000,0.045139,0.045139,0.010417,0.006944,0.000000,0.000000,0.000000,0.010417,...,0.0,0.0,0.0,0.0,0.0,0.010417,0.805556,0.184028,0.00000,0
7,7,0.097500,0.100000,0.015000,0.002500,0.000000,0.000000,0.000000,0.002500,0.127500,...,0.0,0.0,0.0,0.0,0.0,0.115000,0.862500,0.022500,0.00000,0
8,8,0.000000,0.164948,0.034364,0.010309,0.006873,0.000000,0.000000,0.000000,0.068729,...,0.0,0.0,0.0,0.0,0.0,0.134021,0.707904,0.158076,0.00000,0
9,9,0.005181,0.126943,0.119171,0.033679,0.012953,0.000000,0.000000,0.000000,0.010363,...,0.0,0.0,0.0,0.0,0.0,0.049223,0.660622,0.290155,0.00000,0


In [29]:
y_train = data['TARGET']

X_train = sp.sparse.csr_matrix(
    data.drop(['index', col_target], axis=1).values
)

In [30]:
del data

In [31]:
X_train.shape, type(X_train), y_train.shape, type(y_train)

((46704, 2088),
 scipy.sparse.csr.csr_matrix,
 (46704,),
 pandas.core.series.Series)

In [35]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression

time_split = TimeSeriesSplit(n_splits=3)

for el in time_split.split(X_train):
    print(el[0].shape, el[1].shape)

(11676,) (11676,)
(23352,) (11676,)
(35028,) (11676,)


In [36]:
logit = LogisticRegression(C=2.3263050671536263, random_state=SEED, solver='saga',
                           class_weight='balanced', penalty='l2', n_jobs = 4)

cv_scores1 = cross_val_score(
    logit, X_train, y_train, cv=time_split, 
    scoring='roc_auc', n_jobs=4
)
print(cv_scores1, cv_scores1.mean())

[0.87367505 0.74792441 0.90266191] 0.8414204570441731


In [27]:
# [0.84263587 0.57724162 0.87377332] 0.7645502695952456  LogisticRegression(C=1, random_state=SEED, solver='liblinear')
# [0.92467118 0.71270885 0.89997972] 0.8457865816276436  LogisticRegression(C=1.467799267622069, random_state=SEED, solver='saga',class_weight='balanced', penalty='l2', n_jobs = 4)
# [0.9191348  0.73272748 0.90692917] 0.8529304836561487  LogisticRegression(C=2.3263050671536263, random_state=SEED, solver='saga', class_weight='balanced', penalty='l2', n_jobs = 4)

In [28]:
c_values = np.logspace(0.15, 0.4, 5)
c_values

array([1.41253754, 1.63117291, 1.88364909, 2.17520403, 2.51188643])

### MODEL WITH GRID

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV

time_split = TimeSeriesSplit(n_splits=3)

param_grid_log = {
    'solver': ['sag', 'saga'],
    'penalty': ['l2'],
    'C': c_values,
    'class_weight': ['balanced']
}

model_log = LogisticRegression(random_state=SEED, n_jobs = 4)

grid = GridSearchCV(
    model_log, param_grid_log, cv=time_split, 
    n_jobs=-1, scoring='roc_auc', verbose=1
)
grid.fit(X_train, y_train)

best_params_ = grid.best_params_
model_ = grid.best_estimator_
score_ = grid.best_score_

print(score_, best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   30.3s finished


0.8528041459772181 {'C': 1.4125375446227544, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'saga'}


In [30]:
# 0.8457865816276435 {'C': 1.467799267622069, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'saga'}

In [31]:
locals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'rn', 'os', 'sys', 'gc', 'np', 'pd', 'scipy', 'plt', 'sns', 'display', 'warnings', 'BASE_DIR', 'inspect', 'datetime', 'display_html', 'set_parameter_csv', 'get_memory_usage', 'null_verificator', 'reduce_size_data', 'add_col_dates', 'analysis_for_group', 'get_default_args', 'corr_detail', 'ratio_convergencia', 'compare_categories', 'display_horizontal', 'serie_ratio_convergencia', 'replace_value_ratio', 'get_seed', 'numeric_to_cat', 'numeric_to_cat_train', 'KMeans', 'silhouette_samples', 'silhouette_score', 'cm', 'graph_analysis_best_number_cluster', 'graphs_analysis', 'plot_2d_point', 'simple_get_siluet_score', 'get_inertia', 'graphs_box_disp', 'flash_analysis', 'flash_numeric', 'SEED', 'subfolder', '_1', '_i2', 'col_user', '_2', '_i3', '_3', '_i4', '_4', '_i5', 'col', '_i

In [33]:
X_test_k_mini['SCORE'] = grid.best_estimator_.predict_proba(X_test_k)[:,1]
X_test_k_mini.head(20)

,PAGE_1,PAGE_2,SCORE
USER_ID,,,
0,0.028300,0.128440,0.057575
1,0.103742,0.157271,0.028627
2,0.032517,0.168074,0.032511
3,0.005168,0.192765,0.024127
4,0.010628,0.120451,0.055247
5,0.000510,0.132188,0.036333
6,0.001827,0.084044,0.056528
7,0.044106,0.176034,0.037222
8,0.001740,0.163549,0.038012


In [37]:
del X_test_k_mini['PAGE_1']
del X_test_k_mini['PAGE_2']
X_test_k_mini.head()

,SCORE
USER_ID,
0,0.057575
1,0.028627
2,0.032511
3,0.024127
4,0.055247


In [39]:
X_test_k_mini.to_csv('intent_1_timeseriesplit.csv')

In [41]:
predict_train = grid.best_estimator_.predict_proba(X_train)[:,1]
predict_train, predict_train.shape

(array([0.07270878, 0.02449329, 0.03410859, ..., 0.09338273, 0.41830093,
        0.06252517]), (46704,))

In [43]:
n = int(46704/4)

11676

In [45]:
a = [1,2,3,4,5,6,7,8,9]
a[0:3], a[3:6], a[6:9]

([1, 2, 3], [4, 5, 6], [7, 8, 9])

In [47]:
predict_train[0*11676 + 0], predict_train[1*11676 + 0], predict_train[2*11676 + 0]

(0.0727087843509679, 0.08145797774542754, 0.057286186248476666)

In [49]:
predict_train[0*11676:1*11676].shape, predict_train[1*11676:2*11676].shape, predict_train[2*11676:3*11676].shape, predict_train[3*11676:4*11676].shape

((11676,), (11676,), (11676,), (11676,))

In [51]:
X_test_k_mini['USER_ID'] = X_test_k_mini.index
del X_test_k_mini.index.name
X_test_k_mini.head()

,SCORE,USER_ID
0,0.057575,0
1,0.028627,1
2,0.032511,2
3,0.024127,3
4,0.055247,4


In [55]:
# EXPERIMENTANDO MUAJAJAJAJAJAJAJJAJAJAJAJ

X_test_k_mini['TRIM_1'] = predict_train[0*11676:1*11676]
X_test_k_mini['TRIM_2'] = predict_train[1*11676:2*11676]
X_test_k_mini['TRIM_3'] = predict_train[2*11676:3*11676]
X_test_k_mini['TRIM_4'] = predict_train[3*11676:4*11676]
X_test_k_mini['TRIM_MEAN'] = (X_test_k_mini['TRIM_1'] + X_test_k_mini['TRIM_2'] + X_test_k_mini['TRIM_3'] + X_test_k_mini['TRIM_4']) / 4
X_test_k_mini['TRIM_MEAN_1_4'] = (X_test_k_mini['TRIM_1'] + X_test_k_mini['TRIM_4']) / 2
X_test_k_mini.head()

,SCORE,USER_ID,TRIM_1,TRIM_2,TRIM_3,TRIM_4,TRIM_MEAN,TRIM_MEAN_1_4
0,0.057575,0,0.072709,0.081458,0.057286,0.031577,0.060758,0.052143
1,0.028627,1,0.024493,0.033227,0.030000,0.027765,0.028871,0.026129
2,0.032511,2,0.034109,0.031641,0.028227,0.038146,0.033031,0.036127
3,0.024127,3,0.018638,0.023581,0.026613,0.027811,0.024161,0.023224
4,0.055247,4,0.072940,0.073549,0.042994,0.028510,0.054498,0.050725


In [56]:
X_test_k_mini[['SCORE', 'TRIM_1', 'TRIM_2', 'TRIM_3', 'TRIM_4', 'TRIM_MEAN', 'TRIM_MEAN_1_4']].corr()

,SCORE,TRIM_1,TRIM_2,TRIM_3,TRIM_4,TRIM_MEAN,TRIM_MEAN_1_4
SCORE,1.000000,0.621122,0.733898,0.333790,0.693461,0.841788,0.806548
TRIM_1,0.621122,1.000000,0.539769,0.198556,0.333169,0.718464,0.799271
TRIM_2,0.733898,0.539769,1.000000,0.243605,0.427094,0.778864,0.589022
TRIM_3,0.333790,0.198556,0.243605,1.000000,0.234918,0.610756,0.266268
TRIM_4,0.693461,0.333169,0.427094,0.234918,1.000000,0.711275,0.832928
TRIM_MEAN,0.841788,0.718464,0.778864,0.610756,0.711275,1.000000,0.875033
TRIM_MEAN_1_4,0.806548,0.799271,0.589022,0.266268,0.832928,0.875033,1.000000


In [57]:
for val in [1, 2, 3, 4, 'MEAN', 'MEAN_1_4']:
    col_predict = 'TRIM_{}'.format(val)
    
    X_test_k_mini[['USER_ID', col_predict]].rename(
        columns={col_predict: 'SCORE'}
    ).to_csv(
        'init_attemp_{}.csv'.format(val), 
        index=False
    )